This is v1 of the control law propagator
There will be three main modules:

- 1) Propagator

- 2) sliding window

- 3) window filter

In [1]:
import numpy as np
import scipy as sp
import ode

In [45]:
# Inputs:
q_0 = np.array([0,2])
p_0 = np.array([1,4])
u_0 = np.array([2])

t_start = 10
t_end =  11

integrateTol = 10**-3
integrateMaxIter = 40
state_dim = 2
Gamma = 1

In [46]:
np.ones(np.shape(q_0))

array([1., 1.])

In [47]:
def H_T_p(q,p,u):
    # for q-dot
    return np.ones(np.shape(q))+q

def H_T_q(q,p,u):
    # for p-dot
    return np.ones(np.shape(p))+p
    
def Q_u(q,p,u):
    # for u-dot
    return np.ones(np.shape(u))+u
    

In [48]:
def rhs(t, qpu_vec, **kwargs):
    '''
    Input: qpu_vec
    
    Output: qpu_vec
    '''
    state_dim = kwargs['state_dim']
    Gamma = kwargs['Gamma']
    q = qpu_vec[:state_dim]
    p = qpu_vec[state_dim:2*state_dim]
    u = qpu_vec[2*state_dim:]
    q_dot =  H_T_p(q,p,u)
    p_dot = -1*H_T_q(q,p,u)
    u_dot = -Gamma*Q_u(q,p,u)
    return np.hstack([q_dot, p_dot, u_dot])

In [49]:
print np.hstack([q_0, p_0, u_0])[2:]
print np.hstack([q_0, p_0, u_0])[2:2*2]
print np.hstack([q_0, p_0, u_0])

[1 4 2]
[1 4]
[0 2 1 4 2]


In [50]:
# call the ode to integrate the dynamic equations to get q, p, and u

ODE takes as input:
- RHS
- initial condition
- t_start and t_end for which you want your desired output values

In [51]:
qpu_vec = np.hstack([q_0, p_0, u_0])
qpu_vec_i, t, failFlag, iter_i = ode.ode_rk23(rhs, t_start, t_end, qpu_vec, integrateTol, integrateMaxIter, state_dim=state_dim, Gamma = Gamma)

In [52]:
# only take last value
qpu_vec_i[-1]

array([ 1.71656809,  7.14970426, -0.26496265,  0.83759338,  0.10255603])

In [76]:
range(0,20,(20-0)/2)

[0, 10]

In [198]:
# inputs
t_0 = 0
T = 4
K = 2
# print range(t_0,T,(T-t_0)/(2*K))

# end inputs
qs=[]
ps=[]
us=[]
qpu_vec = np.hstack([q_0, p_0, u_0])
ts = range(t_0,T+1,(T-t_0)/(2*K)) # go until T+1 because last value will be used as starting point for next window
print ts
for i in range(len(ts)-1):
    t_start, t_end = ts[i], ts[i+1]
    qpu_vec_i, t, failFlag, iter_i = ode.ode_rk23(rhs, t_start, t_end, qpu_vec, integrateTol, integrateMaxIter, state_dim=state_dim, Gamma = Gamma)
    qpu_vec = qpu_vec_i[-1] # only need the last value
    if i == len(ts)-2 :
        pass 
        # no need to append since weight = 0 for last value
    else:
        qs.append(qpu_vec[:state_dim])
        ps.append(qpu_vec[state_dim:2*state_dim])
        us.append(qpu_vec[2*state_dim:])
        

[0, 1, 2, 3, 4]


In [199]:
qs

[array([1.71656809, 7.14970426]),
 array([ 6.37867051, 21.13601153]),
 array([19.03713547, 59.1114064 ])]

In [200]:
ps

[array([-0.26496265,  0.83759338]),
 array([-0.72993435, -0.32483587]),
 array([-0.90081606, -0.75204015])]

In [203]:
us

[array([0.10255603]), array([-0.59490152]), array([-0.85122409])]

In [204]:
qpu_vec

array([ 53.4069695 , 162.22090851,  -0.96358004,  -0.90895009,
        -0.94537005])

### Triangle window filter

In [155]:
print K
weights_0 = [float(i)/K for i in range(1,K+1)]  
weights_1 = [2-(float(i)/K) for i in range(K+1,(2*K)+1)]
# sanity check 
assert len(weights_0)==len(weights_1)
weights = weights_0+weights_1

2


In [156]:
print weights_0
print weights_1

[0.5, 1.0]
[0.5, 0.0]


In [211]:
weights_total = sum(weights[:-1])
print weights_total

for vec in [qs, ps, us]:
    qpus_weighted = [val*w for val,w in zip(vec, weights[:-1])]
    print qpus_weighted
    qpus_normalized = [vec/weights_total for vec in qpus_weighted]
    print qpus_normalized
#     print qpus_weighted/weights_total

#TODO: still need a way to package all the qs, ps, us into one object to return 

2.0
[array([0.85828404, 3.57485213]), array([ 6.37867051, 21.13601153]), array([ 9.51856773, 29.5557032 ])]
[array([0.42914202, 1.78742606]), array([ 3.18933526, 10.56800577]), array([ 4.75928387, 14.7778516 ])]
[array([-0.13248132,  0.41879669]), array([-0.72993435, -0.32483587]), array([-0.45040803, -0.37602008])]
[array([-0.06624066,  0.20939835]), array([-0.36496717, -0.16241794]), array([-0.22520402, -0.18801004])]
[array([0.05127801]), array([-0.59490152]), array([-0.42561205])]
[array([0.02563901]), array([-0.29745076]), array([-0.21280602])]
